In [ ]:
import pandas as pd
import humanize

In [ ]:
rb = pd.read_csv('../../tmp-data/ss_range_bars.final.csv', parse_dates=['timestamp'], index_col='timestamp')
rb.columns

In [ ]:
signal_only_df = rb.drop(columns=['Open', 'High', 'Low', 'Close', 'volume', 'macd', 'macd_signal', 'macd_histogram', 'bb_upper', 'bb_lower',
       'bb_distance', 'rsi'])
signal_only_df

In [ ]:
signal_count = signal_only_df['signal'].value_counts()
signal_count 

In [ ]:
kline_df = pd.read_csv('../../data/btcusdt-1year-1m-merged.csv', usecols=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
kline_df['date'] = pd.to_datetime(kline_df['timestamp']).dt.date 
kline_df['timestamp'] = pd.to_datetime(kline_df['timestamp'])
kline_df.set_index('timestamp', inplace=True)
kline_df.sort_index(inplace=True)
kline_df.rename(columns={'close': 'Close', 'open': 'Open', 'high': 'High', 'low': 'Low'}, inplace=True)
kline_df

In [ ]:
merged_df = pd.merge(kline_df, signal_only_df, left_index=True, right_index=True, how='left')
merged_df['signal'].fillna(value=0, inplace=True)
merged_df

In [ ]:
merged_df[(abs(merged_df['signal']) > 0)]

In [ ]:
signal_count = merged_df['signal'].value_counts()
signal_count

In [ ]:
df = merged_df

In [ ]:
import random
from backtesting import Strategy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

class RangeBarStrategyPreCalc(Strategy):

    per_trade_risk = 0.1
    rs_sl_factor = 0.1
    potential_profit_factor = 0.25
    
    def init(self):
       pass

    # Stop Loss Price = Liquidation Price - (Liquidation Price - Entry Price) / Leverage * Stop Loss Ratio
    def next(self):
        ts = self.data.index[-1]
        current_close = self.data.Close[-1]
        pre_calc_signal = self.data.signal[-1]
     
        if pre_calc_signal > 0:
            rs_sl = self.data.average_adr[-1] * self.rs_sl_factor
            potential_profit = self.data.average_adr[-1] * self.potential_profit_factor
            order_placed = True        
            self.buy(size=self.per_trade_risk, sl=current_close - rs_sl, tp=current_close + potential_profit)
        elif pre_calc_signal < 0:
            rs_sl = self.data.average_adr[-1] * self.rs_sl_factor
            potential_profit = self.data.average_adr[-1] * self.potential_profit_factor
            self.sell(size=self.per_trade_risk, sl=current_close + rs_sl , tp=current_close - potential_profit)

In [ ]:
import pyperclip as clip
import tabulate

In [ ]:
from backtesting import Backtest
bt = Backtest(df, RangeBarStrategyPreCalc, hedging=False, cash=100000, margin=1/20, commission=(0.04 / 100), exclusive_orders=False)
stats = bt.run()
md = stats.to_markdown()
clip.copy(md)
stats

## RB Signals Merged With Kline Source

|                        | 0                                                                          |
|:-----------------------|:---------------------------------------------------------------------------|
| Start                  | 2022-04-13 18:27:00                                                        |
| End                    | 2023-04-14 16:39:00                                                        |
| Duration               | 365 days 22:12:00                                                          |
| Exposure Time [%]      | 48.02052632877425                                                          |
| Equity Final [$]       | 67011.0654171556                                                           |
| Equity Peak [$]        | 135369.62312571923                                                         |
| Return [%]             | -32.9889345828444                                                          |
| Buy & Hold Return [%]  | -26.98687433394588                                                         |
| Return (Ann.) [%]      | -32.8425877509108                                                          |
| Volatility (Ann.) [%]  | 38.81420532692596                                                          |
| Sharpe Ratio           | 0.0                                                                        |
| Sortino Ratio          | 0.0                                                                        |
| Calmar Ratio           | 0.0                                                                        |
| Max. Drawdown [%]      | -51.897842498051894                                                        |
| Avg. Drawdown [%]      | -1.8947281385869634                                                        |
| Max. Drawdown Duration | 244 days 12:56:00                                                          |
| Avg. Drawdown Duration | 3 days 07:29:00                                                            |
| # Trades               | 12080                                                                      |
| Win Rate [%]           | 34.38741721854305                                                          |
| Best Trade [%]         | 3.1480288751336127                                                         |
| Worst Trade [%]        | -1.6776672963060735                                                        |
| Avg. Trade [%]         | -0.008570063133961359                                                      |
| Max. Trade Duration    | 1 days 02:58:00                                                            |
| Avg. Trade Duration    | 0 days 01:32:00                                                            |
| Profit Factor          | 0.981646934088045                                                          |
| Expectancy [%]         | -0.005843222028528298                                                      |
| SQN                    | -1.5096543308170662                                                        |
| _strategy              | RangeBarStrategyPreCalc                                                    |
| _equity_curve          | Equity  DrawdownPct  DrawdownDuration                                      |
|                        | timestamp                                                                  |
|                        | 2022-04-13 18:27:00  100000.000000     0.000000               NaT          |
|                        | 2022-04-13 18:28:00  100000.000000     0.000000               NaT          |
|                        | 2022-04-13 18:29:00  100000.000000     0.000000               NaT          |
|                        | 2022-04-13 18:30:00  100000.000000     0.000000               NaT          |
|                        | 2022-04-13 18:31:00  100000.000000     0.000000               NaT          |
|                        | ...                            ...          ...               ...          |
|                        | 2023-04-14 16:35:00   67011.065417     0.504977               NaT          |
|                        | 2023-04-14 16:36:00   67011.065417     0.504977               NaT          |
|                        | 2023-04-14 16:37:00   67011.065417     0.504977               NaT          |
|                        | 2023-04-14 16:38:00   67011.065417     0.504977               NaT          |
|                        | 2023-04-14 16:39:00   67011.065417     0.504977 244 days 12:56:00          |
|                        |                                                                            |
|                        | [526933 rows x 3 columns]                                                  |
| _trades                | Size  EntryBar  ExitBar   EntryPrice     ExitPrice         PnL             |
|                        | 0        -1      1277     1333  39966.00720  39756.128000  209.879200  \   |
|                        | 1        -1      1543     1565  39661.72896  39790.586000 -128.857040      |
|                        | 2         1      1571     1797  39874.24332  40084.272000  210.028680      |
|                        | 3         1      1797     1812  40046.11204  39917.014000 -129.098040      |
|                        | 4        -1      1819     1821  39729.10200  39857.986000 -128.884000      |
|                        | ...     ...       ...      ...          ...           ...         ...      |
|                        | 12075    -1    486429   486639  26442.81864  26642.508571 -199.689931      |
|                        | 12076    -1    486705   486740  26302.17492  26502.408571 -200.233651      |
|                        | 12077     1    486747   486909  26521.60440  26889.617143  368.012743      |
|                        | 12078    -1    500917   500924  26733.70224  26518.014286  215.687954      |
|                        | 12079    -1    502432   502575  26849.75580  26973.692857 -123.937057      |
|                        |                                                                            |
|                        |        ReturnPct           EntryTime            ExitTime        Duration   |
|                        | 0       0.005251 2022-04-14 15:44:00 2022-04-14 16:40:00 0 days 00:56:00   |
|                        | 1      -0.003249 2022-04-14 20:10:00 2022-04-14 20:32:00 0 days 00:22:00   |
|                        | 2       0.005267 2022-04-14 20:38:00 2022-04-15 00:24:00 0 days 03:46:00   |
|                        | 3      -0.003224 2022-04-15 00:24:00 2022-04-15 00:39:00 0 days 00:15:00   |
|                        | 4      -0.003244 2022-04-15 00:46:00 2022-04-15 00:48:00 0 days 00:02:00   |
|                        | ...          ...                 ...                 ...             ...   |
|                        | 12075  -0.007552 2023-03-17 13:36:00 2023-03-17 17:06:00 0 days 03:30:00   |
|                        | 12076  -0.007613 2023-03-17 18:12:00 2023-03-17 18:47:00 0 days 00:35:00   |
|                        | 12077   0.013876 2023-03-17 18:54:00 2023-03-17 21:36:00 0 days 02:42:00   |
|                        | 12078   0.008068 2023-03-27 15:04:00 2023-03-27 15:11:00 0 days 00:07:00   |
|                        | 12079  -0.004616 2023-03-28 16:19:00 2023-03-28 18:42:00 0 days 02:23:00   |
|                        |                                                                            |
|                        | [12080 rows x 10 columns]                                                  |

## RB Final - erroneous rb creation

|                        | 0                                                                          |
|:-----------------------|:---------------------------------------------------------------------------|
| Start                  | 2022-04-13 18:27:00                                                        |
| End                    | 2023-04-14 16:39:00                                                        |
| Duration               | 365 days 22:12:00                                                          |
| Exposure Time [%]      | 43.57198029489006                                                          |
| Equity Final [$]       | 20909558096.570538                                                         |
| Equity Peak [$]        | 20909558096.570538                                                         |
| Return [%]             | 20909458.096570536                                                         |
| Buy & Hold Return [%]  | -27.154721765685903                                                        |
| Return (Ann.) [%]      | 19559102.794789784                                                         |
| Volatility (Ann.) [%]  | 31059145.45268928                                                          |
| Sharpe Ratio           | 0.6297373127854761                                                         |
| Sortino Ratio          | 1235888.12045257                                                           |
| Calmar Ratio           | 1690232.5200632848                                                         |
| Max. Drawdown [%]      | -11.571841484896684                                                        |
| Avg. Drawdown [%]      | -1.3581442860286381                                                        |
| Max. Drawdown Duration | 10 days 19:02:00                                                           |
| Avg. Drawdown Duration | 0 days 07:21:00                                                            |
| # Trades               | 14076                                                                      |
| Win Rate [%]           | 37.091503267973856                                                         |
| Best Trade [%]         | 4.748848942760664                                                          |
| Worst Trade [%]        | -3.476291644907592                                                         |
| Avg. Trade [%]         | 0.2571517195338213                                                         |
| Max. Trade Duration    | 1 days 03:28:00                                                            |
| Avg. Trade Duration    | 0 days 01:31:00                                                            |
| Profit Factor          | 2.776746195749678                                                          |
| Expectancy [%]         | 0.2603769491897146                                                         |
| SQN                    | 14.058428578942353                                                         |
| _strategy              | RangeBarStrategyPreCalc                                                    |
| _equity_curve          | Equity  DrawdownPct DrawdownDuration                                       |
|                        | timestamp                                                                  |
|                        | 2022-04-13 18:27:00  1.000000e+05     0.000000              NaT            |
|                        | 2022-04-13 18:28:00  1.000000e+05     0.000000              NaT            |
|                        | 2022-04-13 18:29:00  1.000000e+05     0.000000              NaT            |
|                        | 2022-04-13 18:30:00  1.000000e+05     0.000000              NaT            |
|                        | 2022-04-13 18:31:00  1.000000e+05     0.000000              NaT            |
|                        | ...                           ...          ...              ...            |
|                        | 2023-04-14 16:35:00  2.075028e+10     0.005616              NaT            |
|                        | 2023-04-14 16:36:00  2.075028e+10     0.005616              NaT            |
|                        | 2023-04-14 16:37:00  2.090956e+10     0.000000  0 days 09:45:00            |
|                        | 2023-04-14 16:37:01  2.090956e+10     0.000000              NaT            |
|                        | 2023-04-14 16:39:00  2.090956e+10     0.000000              NaT            |
|                        |                                                                            |
|                        | [291295 rows x 3 columns]                                                  |
| _trades                | Size  EntryBar  ExitBar    EntryPrice  ExitPrice           PnL             |
|                        | 0          -1       875      911  39893.564189  39796.542  9.702219e+01  \ |
|                        | 1          -1      1035     1044  39646.648994  39775.500 -1.288510e+02    |
|                        | 2           1      1051     1051  39665.960040  39650.100 -1.586004e+01    |
|                        | 3           1      1052     1052  39665.960040  39650.100 -1.586004e+01    |
|                        | 4           1      1200     1204  39894.851560  40217.858  3.230064e+02    |
|                        | ...       ...       ...      ...           ...        ...           ...    |
|                        | 14071 -253001    291248   291248  30239.491363  30251.592 -3.061473e+06    |
|                        | 14072 -260722    291247   291292  30131.480585  30035.484  2.502842e+07    |
|                        | 14073 -252260    291245   291292  30239.491363  30035.484  5.146290e+07    |
|                        | 14074 -257452    291244   291292  30239.491363  30035.484  5.252210e+07    |
|                        | 14075 -262752    291243   291292  30239.491363  30035.484  5.360334e+07    |
|                        |                                                                            |
|                        |        ReturnPct           EntryTime            ExitTime        Duration   |
|                        | 0       0.002432 2022-04-14 15:50:00 2022-04-14 16:36:00 0 days 00:46:00   |
|                        | 1      -0.003250 2022-04-14 20:10:00 2022-04-14 20:31:00 0 days 00:21:00   |
|                        | 2      -0.000400 2022-04-14 20:38:00 2022-04-14 20:38:00 0 days 00:00:00   |
|                        | 3      -0.000400 2022-04-14 20:39:00 2022-04-14 20:39:00 0 days 00:00:00   |
|                        | 4       0.008096 2022-04-15 00:22:00 2022-04-15 00:25:00 0 days 00:03:00   |
|                        | ...          ...                 ...                 ...             ...   |
|                        | 14071  -0.000400 2023-04-14 15:32:01 2023-04-14 15:32:01 0 days 00:00:00   |
|                        | 14072   0.003186 2023-04-14 15:32:00 2023-04-14 16:37:00 0 days 01:05:00   |
|                        | 14073   0.006746 2023-04-14 15:30:00 2023-04-14 16:37:00 0 days 01:07:00   |
|                        | 14074   0.006746 2023-04-14 15:29:00 2023-04-14 16:37:00 0 days 01:08:00   |
|                        | 14075   0.006746 2023-04-14 15:28:00 2023-04-14 16:37:00 0 days 01:09:00   |
|                        |                                                                            |
|                        | [14076 rows x 10 columns]                                                  |